## Oppstart

In [56]:
import warnings
warnings.filterwarnings('ignore')

In [57]:
import dhlab.nbtext as nb
import dhlab.token_map as tm
from dhlab.nbtext import get_urn, make_graph, get_urn, relaterte_ord, navn, totals
import dhlab.graph_networkx_louvain as gnl
import pandas as pd
import networkx as nx

%matplotlib inline

plot = lambda x,  lw=3, alpha= 0.7: x.plot(figsize=(15,6), lw=lw, alpha=alpha)


In [58]:
from dhlab.module_update import update, css
css()

# Hjelpekommandoer

I den her notebooken er det satt opp kode for å telle opp navn i et korpus. Både listen med navn og korpuset må foreligge som excelfiler. Noen av kommandoene vil bli lagt til `token_map`-modulen 

* `tell_navn` teller opp navnene på tvers av korpuset 
* `korpus_oppføring` finner data i korpuset
* `vis_bok` viser navn i en spesifikk bok (benytter oppførings-kommandoen)
* `aggregate_pr_col` aggregerer navnedata over kolonner.

In [59]:
import pandas as pd
from nbtext import metadata

def alle_korpusnavn(korpus):
    urner = nb.pure_urn(korpus)
    alle_navn = tm.combine_names(tm.corpus_names(urner))
    return alle_navn

def tell_navn(korpus, token_map):
    res = dict()
    for urn in nb.pure_urn(korpus['urn']):
        try:
            res[urn] = tm.count_name_strings(str(urn), token_map)
            res[urn].columns = [str(urn)]
        except:
            print("Problemer med å telle fra: ", urn)
    return pd.concat([res[u] for u in res], axis=1)

def korpus_oppføring(df, text, column='tittel'):
    return nb.pure_urn(df[df[column].str.contains(text)]['urn'])

def add_metadata(korpus, how='inner'):
    f = nb.frame([(z[0], '_'.join([str(x) for x in z[2:4]])) for z in nb.metadata(korpus['urn'])], 'urn metadata'.split())
    f = f.astype({'urn':'str'})
    return korpus.merge(f, on = 'urn', how = how)
    
vis_bok = lambda x, column = 'tittel': nb.frame_sort(nb.frame(opptelling[korpus_oppføring(korpus, x, column)[0]].dropna(), nb.metadata(korpus_oppføring(korpus, x, column)[0])[0][3]))

    

def bygg_allgraf(korpus):
    graphs = dict()
    for u in korpus.urn:
        graphs[u] = tm.character_network(u, redigerte_navn)
    df_graphs = [nx.to_pandas_adjacency(graphs[x]) for x in graphs]
    g = df_graphs[0]
    for z in df_graphs[1:]:
        g = g.add(z, fill_value=0).fillna(0)

    total_graph = nx.from_pandas_adjacency(g)
    return total_graph

## Navn i bøker


Vi søker etter navn i en bok, og en serie av bøker.

Kommandoene er:
1. `names` henter et sett med forslag til navn, fra nbtext
1. `show_names` gir en oversikt over funnene, fra token_map
1. `names_to_token_map_file` lagrer navneforslagene til en excelfil (eller csv) for redigering (token_map)
1. `read_token_map_file` henter redigert fil tilbake for analyse (token_map)
1. `character_network` lager graf for navnene (token_map)
1. `show_graph`  tegner grafen (fra graph_networkx_louvain)
1. `show_communities` viser clustre i grafen (graph_networkx_louvain)



## Korpus

Hent korpus fra excelfil. Det er korpuset det skal telles opp fra. Korpuset som genererer navn kan godt være forskjellig fra der det telles.

In [60]:
korpus = nb.restore_metadata_from_excel('Alt_Hamsunkorpus.xlsx')
korpus

,urn,forf,år,tittel,undertittel,forlag,sjanger,sesamid,Unnamed: 8,Unnamed: 9
0,2008112113001,"Hamsun, Knut",1877,Den Gaadefulde,en Kjærlighedshistorie fra Nordland,Urdal,unknown,05b5fb8e2ffab70be305373f9857ee85,NaN,NaN
1,2009061013011,"Hamsun, Knut",1878,Bjørger,fortælling,[K. Hamsund];Alb.Fr. Knudsen,unknown,709614fcb48740925154af4130d7a26c,NaN,NaN
2,2009061013012,"Hamsun, Knut",1878,Et gjensyn,NaN,[s.n.];Alb. Fr. Knudsen,unknown,233ad96c3f3bedf1739ceb29a81534d4,NaN,NaN
3,2015051929001,"Hamsun, Knut",1890,Sult,NaN,Philipsen,novel,b4a0a4c7765fa29eb4a1ba910405806e,NaN,NaN
4,2008123012002,"Hamsun, Knut",1892,Mysterier,Roman,Philipsen,unknown,9f41e621267dd5475db9b7e45cd00367,NaN,NaN
5,2008123012003,"Hamsun, Knut",1893,Redaktør Lynge,Roman,Philipsen,fiction,e09ea1ee062affda9ab9fdfde810517c,NaN,NaN
6,2008123010002,"Hamsun, Knut",1893,Ny Jord,Roman,Philipsen,unknown,aca8b7840508f729920970a51fea6c05,NaN,NaN
7,2008082710002,"Hamsun, Knut",1894,Pan,af Løjtnant Thomas Glahns Papirer,Philipsen,unknown,8dab706553bacde19fa1a30c06896464,NaN,NaN
8,2008123010003,"Hamsun, Knut",1895,Ved Rigets Port,Forspil,Philipsen,unknown,c8af50e6f92caeaa7742b651358e2b14,NaN,NaN
9,2009010210001,"Hamsun, Knut",1896,Livets Spil,NaN,Det Nordiske Forlag,unknown,8637d081c40a6df2c59789f422b08c11,NaN,NaN


## Sjekk innholdet

Gjør noen søk for å se at alt er ok. Konkordanser er fine til det. 

In [61]:
nb.urn_concordance(word = 'Ganges', urns = korpus) 

"Kratskog, Hamsun, Knut, 1903",. Kanske ved Bredden af,Ganges,hvor Lotosen aldrig visned .
"Men Livet lever, Hamsun, Knut, 1933","det var Sus , var",Ganges,"og Amazonflod , Suset blev"
"Samlede verker, Hamsun, Knut, 2009","Syden , til breddene av",Ganges,der menneskets første vugge stod
"Samlede verker, Hamsun, Knut, 2009","land , til bredden av",Ganges,hvor de første menneskers vugge
"Samlede verker, Hamsun, Knut, 2009","land , til bredden av",Ganges,hvor de første menneskers vugge


### Hent redigert fil

Husk at verdien på orient må være lik mellom lagring og lesing av excelfil. Typisk for den her type redigering er radorientering. Excel har en begrensning på 256 kolonner, som blir for smått for de fleste anvendelser. Det er tilnærmet ubegrenset kapasitet på antall rader.

In [62]:
redigerte_navn = tm.read_token_map_file('samledestedsnavn_Hamsunkorpus.xls', orient = "row")

## Tell opp navn med hensyn til det som står i `redigerte_navn`

Bruker hjelpekommando `tell_navn`. Det kan ta litt tid å telle opp, et par minutter.

In [63]:
opptelling = tell_navn(korpus, redigerte_navn)

Sånn ser opptellingen ut, en kolonne pr. bok. 

In [64]:
opptelling.head()

,2008112113001,2009061013011,2009061013012,2015051929001,2008123012002,2008123012003,2008123010002,2008082710002,2008123010003,2009010210001,...,2008091204050,2010031603013,2009032400042,2008010400002,2008112804029,2008110500029,2015090706177,2014090308131,2009061012004,2009042313001
Aabakken,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aas,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,71.0,2.0,18.0,1.0,20.0,NaN,NaN,NaN,NaN
Afrika,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,2.0,NaN,NaN,6.0,NaN,2.0,2.0,3.0,NaN
Amerika,NaN,NaN,NaN,NaN,2.0,9.0,5.0,NaN,NaN,NaN,...,39.0,27.0,66.0,133.0,50.0,26.0,121.0,61.0,307.0,4.0
Ananur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Opptelling totalt for korpuset, summering over alle bøkene.

In [65]:
nb.frame_sort(nb.frame(opptelling.sum(axis=1)))

,0
Christiania,2029.0
Byen,1633.0
Verden,1348.0
Nørholmen,1280.0
Norge,1168.0
Amerika,1104.0
Grimstad,716.0
Jorden,714.0
Gaarden,713.0
Tyskland,685.0


In [66]:
steder = nb.frame_sort(nb.frame(opptelling.sum(axis=1)))
steder.to_dict()[0]

{'Aabakken': 20.0,
 'Aas': 128.0,
 'Afrika': 31.0,
 'Amerika': 1104.0,
 'Ananur': 7.0,
 'Argentina': 21.0,
 'Armenia': 7.0,
 'Asien': 22.0,
 'Asker': 29.0,
 'Atlanta': 3.0,
 'Atlanterhavet': 27.0,
 'Aurdal': 155.0,
 'Australien': 33.0,
 'Baku': 40.0,
 'Balberg': 18.0,
 'Baltimore': 1.0,
 'Batum': 6.0,
 'Bayern': 5.0,
 'Belgien': 40.0,
 'Bergen': 426.0,
 'Berlin': 227.0,
 'Bjærget': 38.0,
 'Bodø': 102.0,
 'Bokhara': 15.0,
 'Bolivia': 9.0,
 'Boston': 26.0,
 'Brasil': 2.0,
 'Breidablik': 48.0,
 'Brevik': 6.0,
 'Bryne': 2.0,
 'Byen': 1633.0,
 'Bygdø': 28.0,
 'California': 5.0,
 'Chicago': 72.0,
 'Christiania': 2029.0,
 'Cordilleras': 4.0,
 'Dakota': 19.0,
 'Danmark': 250.0,
 'Danzig': 5.0,
 'Doppen': 209.0,
 'Drammen': 40.0,
 'Drøbak': 82.0,
 'Dubrovnik': 20.0,
 'Egersund': 18.0,
 'Egypten': 23.0,
 'Elverum': 24.0,
 'England': 404.0,
 'Equador': 1.0,
 'Europa': 213.0,
 'Finland': 138.0,
 'Finmarken': 40.0,
 'Fjell': 20.0,
 'Flandern': 2.0,
 'Florø': 4.0,
 'Fosenlandet': 40.0,
 'Frankrike':

# Bygger grafer for alle bøkene

Benytt hjelpekommandoen `bygg_allgraf` for å samle grafene i en stor total graf.

In [67]:
korpusgraf = bygg_allgraf(korpus)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Visualiser og analyser på vanlig måte.

In [ ]:
gnl.show_graph(korpusgraf, spread= 1.9)

Forslag til clustre basert på grafen. Clusteralgoritmen for Louvain kan synes å være litt random, så resultatene kan varierere mellom kjøringer. Men det ser skapelig ut, og gir mening.

In [ ]:
gnl.show_communities(korpusgraf)